# Optimization

Tests to determine memory allocation and performance of varaious functions, looks for bottle necks. 

## Quadrature

### Updating quadrature scheme

In [139]:
## initialize quadrature
include("MvGaussHermite.jl")
m = 5
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)

# test updating 
MvGaussHermite.update_old!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])
@time for i in 1:10 MvGaussHermite.update_old!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0]) end


  0.000226 seconds (1.76 k allocations: 157.500 KiB)


In [140]:
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)
MvGaussHermite.update!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])
@time for i in 1:10 MvGaussHermite.update!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])end


  0.000238 seconds (1.22 k allocations: 118.594 KiB)


### Evaluating quadrature

In [155]:
include("MvGaussHermite.jl")
m = 10
f  = x -> sin(x[1]*x[2])
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)
MvGaussHermite.expected_value(f, Quad)
@time MvGaussHermite.expected_value(f, Quad)

  0.000034 seconds (16 allocations: 1.938 KiB)


1.6263032587282567e-18

## Updating beleif states


In [16]:
include("MvGaussHermite.jl")
include("BellmanOpperators.jl")
include("Examples.jl")
# set example functions for test s
pars = Examples.unknown_growth_rate_pars

T_! = (x,f) -> Examples.unknown_growth_rate_T!(x,f,pars)
T_ = (x,f) -> Examples.unknown_growth_rate_T(x,f,pars)
R = (x,f,obs) -> Examples.unknown_growth_rate_R(x,f,obs,pars)
R_obs = (x,f) -> Examples.unknown_growth_rate_R_obs(x,f,pars)
Sigma_N = Examples.Sigma_N
H = (x,a,obs) -> Examples.H * x
Sigma_O = (a,obs) -> reshape(Examples.Sigma_O(obs),1,1)
delta = 0.95
actions = Examples.unknown_growth_actions
observations = Examples.unknown_growth_observations

x_hat = [0.0,0.0]
x_cov = [1.0 0; 0 1.0]

m = 6
f  = x -> sin(x[1]*x[2])
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)

print(" ")

### Test observaiton model update
The take away here is to use a linear observaiton model where ever possible. 

In [44]:
H = x -> Examples.H * x
BellmanOpperators.propogate_observation_model(x_hat,x_cov,H,Quad)
MvGaussHermite.update!(Quad, x_hat, x_cov)
@time BellmanOpperators.propogate_observation_model(x_hat,x_cov,H,Quad)
@time MvGaussHermite.update!(Quad, x_hat, x_cov)

  0.000090 seconds (453 allocations: 42.781 KiB)
  0.000040 seconds (145 allocations: 13.922 KiB)


2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [45]:
H = Examples.H 
BellmanOpperators.propogate_observation_model(x_hat,x_cov,H)
@time BellmanOpperators.propogate_observation_model(x_hat,x_cov,H)

  0.000011 seconds (4 allocations: 320 bytes)


([0.0], [1.0])

### Test bellman opperator for bottle necks 
 The bottle neck is very clearly evaluating the value function

In [492]:
include("MvGaussHermite.jl")
include("BellmanOpperators.jl")
include("ValueFunctions.jl")
include("Examples.jl")
include("POMDPs.jl")
using KalmanFilters
# initianize POMDP object
pars = Examples.unknown_growth_rate_pars
T_! = (x,f) -> Examples.unknown_growth_rate_T!(x,f,pars)
T_ = (x,f) -> Examples.unknown_growth_rate_T(x,f,pars)
R = (x,f,obs) -> 0.5*exp(x[1]+f[1]) - 1/obs[1]
R_obs = (x,f) -> Examples.unknown_growth_rate_R_obs(x,f,pars)
Sigma_N = Examples.Sigma_N
H = (x,a,obs) -> Examples.H * x
Sigma_O = (a,obs) -> reshape(Examples.Sigma_O(obs),1,1)
delta = 0.95
unknownGrowthRate = POMDPs.init(T_!,T_,R,R_obs,Examples.H,Sigma_N,Sigma_O,delta, [[0.05],[0.1],[0.15]], [[0.05],[1.0]])

print(" ")

In [493]:

# state 
act = [0.15]
obs = [1.0]

# value function 
function test1!(mu)
    return 0.1*mu[2] * mu[1]^2
end 


function test2!(mu, cov)
    v = test1!(mu)
    return v - 0.1*cov[1,1] -0.1*cov[2,2] + 0.05*cov[2,1] +0.75*cov[1,1]*mu[1]
end 

m1 = 10
m2 = 7
Vf = ValueFunctions.init_adjGausianBeleifsInterp(m1, m2, [-5.0,-5.0],[5.0,5.0])

v1 = 1.0*zeros(m1^2)
v2 = 1.0*zeros(m2^5)
v_mu = broadcast(i->test1!(Vf.baseValue.grid[i]), 1:m1^2)
v_dsn = broadcast(i->test2!(Vf.uncertantyAdjustment.nodes[i][1],Vf.uncertantyAdjustment.nodes[i][2]), 1:m2^5)

ValueFunctions.update_base!(Vf,v_mu)
ValueFunctions.update_adjustment!(Vf, v_dsn)


x_hat = [0.4, 0.0]
x_cov = [0.2 0.0; 0.0 0.1]
s = (x_hat,x_cov)


data = BellmanOpperators.init_bellmanIntermidiate(2, 1, 4,4)
print(" ")

LoadError: [91mDomainError with -4.938441702975689:[39m
[91macos(x) not defined for |x| > 1[39m

In [494]:

function compute_coefs(d,m,values,alpha,T_alpha_i_grid)
    # get size of 
    #@assert length(values) == m^d
    #@assert all(size(values) .== m)
    # get constants for each term
    d_bar = broadcast(alpha_i -> sum(alpha_i .> 0), alpha)
    c = 2 .^ d_bar ./(m^d)
    # compute sum for each term
    #z = collect_z(m,d) 
    # this is complicated. it initally broadcasts T_alpha_i over the grid z and takes the dot 
    # product with values array. It then broad casts this function over each set of valeus in alpha
    #vT_sums = broadcast(alpha_i -> sum(broadcast(x -> T_alpha_i(alpha_i,x), z).*values), alpha)   
    
    vT_sums = broadcast(alpha_i -> sum(T_alpha_i_grid.*values), alpha)  
    return c.*vT_sums
end

compute_coefs (generic function with 8 methods)

In [495]:
utils.collect_z(5,2)

25-element Array{Array{Float64,1},1}:
 [-0.9510565162951535, -0.9510565162951535]
 [-0.5877852522924731, -0.9510565162951535]
 [-6.123233995736766e-17, -0.9510565162951535]
 [0.587785252292473, -0.9510565162951535]
 [0.9510565162951535, -0.9510565162951535]
 [-0.9510565162951535, -0.5877852522924731]
 [-0.5877852522924731, -0.5877852522924731]
 [-6.123233995736766e-17, -0.5877852522924731]
 [0.587785252292473, -0.5877852522924731]
 [0.9510565162951535, -0.5877852522924731]
 [-0.9510565162951535, -6.123233995736766e-17]
 [-0.5877852522924731, -6.123233995736766e-17]
 [-6.123233995736766e-17, -6.123233995736766e-17]
 [0.587785252292473, -6.123233995736766e-17]
 [0.9510565162951535, -6.123233995736766e-17]
 [-0.9510565162951535, 0.587785252292473]
 [-0.5877852522924731, 0.587785252292473]
 [-6.123233995736766e-17, 0.587785252292473]
 [0.587785252292473, 0.587785252292473]
 [0.9510565162951535, 0.587785252292473]
 [-0.9510565162951535, 0.9510565162951535]
 [-0.5877852522924731, 0.951056516

In [496]:
function f1(s, a, obs, data, POMDP, V)
    data.x_hat, data.x_cov = s
    tu = KalmanFilters.time_update(data.x_hat, data.x_cov, x ->POMDP.T(x,a),  POMDP.Sigma_N)
    data.x_hat, data.x_cov = KalmanFilters.get_state(tu), KalmanFilters.get_covariance(tu)
    
    #observaton quadrature nodes 
   #H = x -> POMDP.H(x,a,obs) # define measurment function # allocation? 
    data.y_hat, data.y_cov = BellmanOpperators.propogate_observation_model(data.x_hat,data.x_cov,POMDP.H) # allocation 
    data.y_cov .+= POMDP.Sigma_O(a,obs)
    MvGaussHermite.update!(data.Quad_y,data.y_hat,data.y_cov)
    data.new_states_mat = broadcast(y -> BellmanOpperators.new_state(y, data.x_hat,data.x_cov, POMDP.H, POMDP.Sigma_O(a,obs)), data.Quad_y.nodes)
    return 0
end 

function f1_(s, a, obs, data, POMDP, V)
    data.x_hat, data.x_cov = s
    tu = KalmanFilters.time_update(data.x_hat, data.x_cov, x ->POMDP.T(x,a),  POMDP.Sigma_N)
    data.x_hat, data.x_cov = KalmanFilters.get_state(tu), KalmanFilters.get_covariance(tu)
    
    #observaton quadrature nodes 
    H = x -> POMDP.H*x # define measurment function # allocation? 
    data.y_hat, data.y_cov = BellmanOpperators.propogate_observation_model(data.x_hat,data.x_cov,H, data.Quad_x) # allocation 
    data.y_cov .+= POMDP.Sigma_O(a,obs)
    MvGaussHermite.update!(data.Quad_y,data.y_hat,data.y_cov)
    data.new_states_mat = broadcast(y -> BellmanOpperators.new_state(y, data.x_hat,data.x_cov, H, POMDP.Sigma_O(a,obs)), data.Quad_y.nodes)
    return 0
end 


f1_(s,act,obs, data, unknownGrowthRate, V)
@time f1_(s,act,obs, data, unknownGrowthRate, V)

f1(s,act,obs, data, unknownGrowthRate, V)
@time f1(s,act,obs, data, unknownGrowthRate, V)

  0.000273 seconds (526 allocations: 41.875 KiB)
  0.000162 seconds (162 allocations: 12.203 KiB)


0

In [497]:
function f2(s, a, obs, data, POMDP, V)
    data.x_hat, data.x_cov = s
    tu = KalmanFilters.time_update(data.x_hat, data.x_cov, x ->POMDP.T(x,a),  POMDP.Sigma_N)
    data.x_hat, data.x_cov = KalmanFilters.get_state(tu), KalmanFilters.get_covariance(tu)
    
    #observaton quadrature nodes 
    #H = Examples.H # define measurment function # allocation? 
    data.y_hat, data.y_cov = BellmanOpperators.propogate_observation_model(data.x_hat,data.x_cov,POMDP.H) # allocation 
    data.y_cov .+= POMDP.Sigma_O(a,obs)
    MvGaussHermite.update!(data.Quad_y,data.y_hat,data.y_cov)
    data.new_states_mat = broadcast(y -> BellmanOpperators.new_state(y, data.x_hat,data.x_cov, POMDP.H, POMDP.Sigma_O(a,obs)), data.Quad_y.nodes)
    
    data.vals = broadcast(x -> V(data.z,x), data.new_states_mat)
    
    return sum(data.vals .* data.Quad_y.weights)
end 

f2(s,act,obs, data, unknownGrowthRate, Vf)
@time f2(s,act,obs, data, unknownGrowthRate, Vf)

  0.000854 seconds (391 allocations: 49.422 KiB)


-0.03876410703907643


### Test Value functions

In [240]:
all(size(zeros(10)) .== 10)

true

In [68]:
include("ValueFunctions.jl")

Main.ValueFunctions

In [306]:
# value function 
include("ValueFunctions.jl")
function test1!(mu)
    return 0.1*mu[2] * mu[1]^2
end 


function test2!(mu, cov)
    v = test1!(mu)
    return v - 0.1*cov[1,1] -0.1*cov[2,2] + 0.05*cov[2,1] +0.75*cov[1,1]*mu[1]
end 

m1 = 10
m2 = 6
Vf = ValueFunctions.init_adjGausianBeleifsInterp(m1, m2, [-5.0,-5.0],[5.0,5.0])

v1 = 1.0*zeros(m1^2)
v2 = 1.0*zeros(m2^5)
v_mu = broadcast(i->test1!(Vf.baseValue.grid[i]), 1:m1^2)
v_dsn = broadcast(i->test2!(Vf.uncertantyAdjustment.nodes[i][1],Vf.uncertantyAdjustment.nodes[i][2]), 1:m2^5)

ValueFunctions.update_base!(Vf,v_mu)

ValueFunctions.update_adjustment!(Vf, v_dsn)

z = 1.0*zeros(5)
x = ([0.0,0.0],[1.0 0; 0 1.0])
Vf.baseValue(x[1])
@time Vf.baseValue(x[1])

Vf.uncertantyAdjustment(z,x)
@time Vf.uncertantyAdjustment(z,x)

  0.000053 seconds (25 allocations: 1.750 KiB)


  0.000109 seconds (27 allocations: 4.797 KiB)


-0.19976206848347985

### Evaluating the uncertiaty adjustment in the value function is the primary bottle neck in the bellman opperator

Two steps to optimize this process: 
1. Optimize evaluation of chebyshev polynomials in ValueFunctions.jl
2. Down sample the value function i.e use a low order aproximation if it apears to be apropreate.   

In [489]:
include("ValueFunctions.jl")
a = [-1.0,-1.]
b = [1.0,1.]
m = 100
p = ValueFunctions.init_interpolation(a,b,m)
x = [0.0,0.0]
# value function 
function test1!(mu)
    return 0.1*mu[2] * mu[1]^2
end 

v_mu = broadcast(i->test1!(p.grid[i]), 1:length(p.grid))

ValueFunctions.update_interpolation!(p,v_mu)
        
p(x)
@time p(x)


  0.000282 seconds (26 allocations: 41.469 KiB)


3.393707262110219e-16

In [491]:
@time ValueFunctions.update_interpolation!(p,v_mu)


  0.393086 seconds (30.40 k allocations: 393.965 MiB, 31.02% gc time)


5151-element Array{Float64,1}:
  0.0
  0.0
  0.0
 -6.66133814775094e-20
 -2.0816681711721686e-20
  1.0200174038743627e-19
 -4.163336342344337e-20
  1.3877787807814457e-20
  3.5735303605122227e-20
  6.522560269672795e-20
  5.967448757360216e-20
 -2.2898349882893854e-20
 -2.5500435096859066e-20
  ⋮
  5.551115123125783e-20
 -1.0547118733938987e-19
 -1.2168044349891716e-17
 -1.5543122344752191e-19
 -1.1501910535116623e-17
  2.2204460492503132e-20
  3.383515689847627e-16
 -2.220446049250313e-19
  3.383071600637777e-16
 -5.639932965095796e-18
  0.0
  3.273128842356993e-16

In [ ]:
compute_coefs(d,m,values,alpha,T_alpha_i_grid)

In [351]:
include("utils.jl")
utils.T_alpha_i(p.alpha[1],x)
@time utils.T_alpha_i(p.alpha[10],x)

  0.000011 seconds (2 allocations: 48 bytes)


5.51091059616309e-16

In [365]:
include("utils.jl")
v = zeros(length(p.alpha))
utils.T_alpha!(v,x,p.alpha, p.coeficents)
@time utils.T_alpha!(v,x,p.alpha, p.coeficents)

  0.000233 seconds (5 allocations: 80.672 KiB)


3.228315458645783e-16

In [ ]:
0.000285 seconds (3 allocations: 40.344 KiB)
0.000274 seconds (3 allocations: 40.344 KiB)
0.000314 seconds (3 allocations: 40.344 KiB)

In [ ]:

a = [0.0,0.0]
b = [1.0,1.0]
m = 5
d = length(a)
# calcualte nodes
f = n -> -cos((2*n-1)*pi/(2*m))
z = f.(1:m)


nodes = (z.+1)./2 .*transpose(b.-a) #.- a
nodes = mapslices(x-> x .+ a, nodes, dims = 2)

z = (z.+1)./2 .*transpose(repeat([2.0],d))
z = mapslices(x-> x .-  1.0, z, dims = 2)
grid = utils.collect_nodes(nodes) # nodes on desiered domain
grid_mapped = utils.collect_nodes(z) # nodes mapped to -1,1


In [504]:
z

5×5 Array{Float64,2}:
 0.0489435  0.0489435  0.0489435  0.0489435  0.0489435
 0.412215   0.412215   0.412215   0.412215   0.412215
 1.0        1.0        1.0        1.0        1.0
 1.58779    1.58779    1.58779    1.58779    1.58779
 1.95106    1.95106    1.95106    1.95106    1.95106

In [517]:
using IJulia
IJulia.installkernel("Julia 6 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "6",
))

┌ Info: Installing Julia 6 Threads kernelspec in /Users/johnbuckner/Library/Jupyter/kernels/julia-6-threads-1.5
└ @ IJulia /Users/johnbuckner/.julia/packages/IJulia/e8kqU/deps/kspec.jl:94


"/Users/johnbuckner/Library/Jupyter/kernels/julia-6-threads-1.5"

In [3]:
#mport Pkg; Pkg.add("Threads")
#using Threads
a = zeros(10)
Threads.@threads for i = 1:10
    a[i] = Threads.threadid()
end


10-element Array{Float64,1}:
 1.0
 1.0
 2.0
 2.0
 3.0
 3.0
 4.0
 4.0
 5.0
 6.0

In [ ]:
# Initialize model 
include("Examples.jl")
include("ValueFunctions.jl")
include("BeleifMDPSolvers.jl")


pars = Examples.unknown_growth_rate_pars

T_! = (x,f) -> Examples.unknown_growth_rate_T!(x,f,pars)
T_ = (x,f) -> Examples.unknown_growth_rate_T(x,f,pars)
R = (x,f,obs) -> Examples.unknown_growth_rate_R(x,f,obs,pars)
R_obs = (x,f) -> Examples.unknown_growth_rate_R_obs(x,f,pars)
Sigma_N = Examples.Sigma_N
H = (x,a,obs) -> Examples.H * x
Sigma_O = (a,obs) -> reshape(Examples.Sigma_O(obs),1,1)
delta = 0.925
actions = Examples.unknown_growth_actions
observations = Examples.unknown_growth_observations

upper = Examples.unknown_growth_upper
lower = Examples.unknown_growth_lower

solver = BeleifMDPSolvers.init(T_!, T_, R, R_obs, H, Sigma_N, Sigma_O,delta,actions,observations,lower,upper)


include("BeleifMDPSolvers.jl")
BeleifMDPSolvers.solve_observed(solver)
BeleifMDPSolvers.solve(solver)

here0 100.1 

1 73.94595673849162 2 145.86323400899144 3 134.826550872899 4 122.68001713815526 5 155.0330507659942 6 103.09087785261819 7 77.47184370878428 8 99.34651694583995 9 111.2605149994456 10 106.98961980680531 11 103.24132411078077 12 76.17616860246942 13 94.13383220228408 14 106.10820997138146 15 85.60529459025928 16 88.99585477857063 17 71.62356113648808 18 92.29446054372987 19 88.27262702763802 20 96.72169642916077 21 84.21107185909932 22 71.04588016226859 23 59.72446269839281 24 78.90114612302565 25 273.1466333445811 26 85.80915531705666 27 62.29999756754463 28 98.21815991196618 29 78.1042248250199 30 58.61762872772308 31 75.28546609897438 32 62.332081510087434 33 61.65946739264951 34 69.17185885501375 35 74.20927438194617 36 71.53533184927345 37 105.18240209573283 38 74.52692224632308 39 70.78270324038296 40 59.81566514795491 41 70.22997980763934 42 65.9208600046799 43 58.12166036212629 44 54.02600864700974 45 69.20110702136473 46 106.86100369676628 47 111.95249214817271 48 71.075531554